<a href="https://colab.research.google.com/github/stealthOG/Skill_Lab/blob/main/ANN_Forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Artificial Neural Network

#### Ekwe Collins Odinaka


A Neural Network makes prediction through the forward propagation process. An example is shown below, with a neural network having 2 inputs, one hidden layer with 2 nodes, and an output layer with one node.

<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width="600px">


we have 6 weights, 3 bias

In [1]:
import numpy as np

# lets initialize the weights and biases

weights = np.around(np.random.uniform(size=6), decimals=2) # initialize the weights
biases = np.around(np.random.uniform(size=3), decimals=2) # initialize the biases

In [2]:
print(weights)
print(biases)

[0.9  0.48 0.65 0.02 0.4  0.52]
[0.47 0.77 0.37]


In [4]:
# create the input variables, x_1, and x_2
x_1 = 0.5
x_2 = 0.85

In [5]:
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

The weighted sum of the inputs at the first node in the hidden layer is 1.3279999999999998


In [6]:
z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]

print('The weighted sum of the inputs at the second node in the hidden layer is {}'.format(z_12))

The weighted sum of the inputs at the second node in the hidden layer is 1.112


Next, assuming a sigmoid activation function, let's compute the activation of the first node, $a_{1, 1}$, in the hidden layer.


In [7]:
a_11 = 1.0 / (1.0 + np.exp(-z_11))

print('The activation of the first node in the hidden layer is {}'.format(np.around(a_11, decimals=4)))

The activation of the first node in the hidden layer is 0.7905


In [8]:
a_12 = 1.0 / (1.0 + np.exp(-z_12))

print('The activation of the second node in the hidden layer is {}'.format(np.around(a_12, decimals=4)))

The activation of the second node in the hidden layer is 0.7525


Now these activations will serve as the inputs to the output layer. So, let's compute the weighted sum of these inputs to the node in the output layer. Assign the value to **z_2**.


In [9]:
z_2 = a_11 * weights[4] + a_12 * weights[5] + biases[2]

print('The weighted sum of the inputs at the node in the output layer is {}'.format(np.around(z_2, decimals=4)))

The weighted sum of the inputs at the node in the output layer is 1.0775


In [10]:
a_2 = 1.0 / (1.0 + np.exp(-z_2))

print('The output of the network for x1 = 0.5 and x2 = 0.85 is {}'.format(np.around(a_2, decimals=4)))

The output of the network for x1 = 0.5 and x2 = 0.85 is 0.746


Obviously, neural networks for real problems are composed of many hidden layers and many more nodes in each layer. So, we can't continue making predictions using this very inefficient approach of computing the weighted sum at each node and the activation of each node manually. 

In order to code an automatic way of making predictions, let's generalize our network. A general network would take $n$ inputs, would have many hidden layers, each hidden layer having $m$ nodes, and would have an output layer. Although the network is showing one hidden layer, but we will code the network to have many hidden layers. Similarly, although the network shows an output layer with one node, we will code the network to have more than one node in the output layer.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width="600px">


Let's start by defining the structure of the network

##### Creating an ANN Structure

In [11]:
n = 2 # number of inputs
num_hidden_layers = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer

Now that we defined the structure of the network, let's go ahead and inititailize the weights and the biases in the network to random numbers. In order to be able to initialize the weights and the biases to random numbers, we will need to use the **Numpy** library.

In [12]:
num_nodes_previous = n # number of nodes in the previous layer

network = {} # initialize network an an empty dictionary

# loop through each layer and randomly initialize the weights and biases associated with each node
# notice how we are adding 1 to the number of hidden layers in order to include the output layer
for layer in range(num_hidden_layers + 1): 
    
    # determine name of layer
    if layer == num_hidden_layers:
        layer_name = 'output'
        num_nodes = num_nodes_output
    else:
        layer_name = 'layer_{}'.format(layer + 1)
        num_nodes = m[layer]
    
    # initialize weights and biases associated with each node in the current layer
    network[layer_name] = {}
    for node in range(num_nodes):
        node_name = 'node_{}'.format(node+1)
        network[layer_name][node_name] = {
            'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
            'bias': np.around(np.random.uniform(size=1), decimals=2),
        }
    
    num_nodes_previous = num_nodes
    
print(network) # print network

{'layer_1': {'node_1': {'weights': array([0.19, 0.78]), 'bias': array([0.64])}, 'node_2': {'weights': array([0.49, 0.05]), 'bias': array([0.55])}}, 'layer_2': {'node_1': {'weights': array([0.23, 0.72]), 'bias': array([0.55])}, 'node_2': {'weights': array([0.4 , 0.33]), 'bias': array([0.25])}}, 'output': {'node_1': {'weights': array([0.94, 0.28]), 'bias': array([0.39])}}}


In [13]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):
        
        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes

    return network # return the network

Use the *initialize_network* function to create a network that:

1. takes 5 inputs
2. has three hidden layers
3. has 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer
4. has 1 node in the output layer

Call the network **small_network**.


In [14]:
small_network = initialize_network(5, 3, [3, 2, 3], 1)
small_network

{'layer_1': {'node_1': {'weights': array([0.06, 0.62, 0.36, 0.23, 0.91]),
   'bias': array([0.7])},
  'node_2': {'weights': array([0.95, 0.28, 0.16, 0.04, 0.05]),
   'bias': array([0.87])},
  'node_3': {'weights': array([0.39, 0.39, 0.04, 0.11, 0.31]),
   'bias': array([0.77])}},
 'layer_2': {'node_1': {'weights': array([0.34, 0.36, 0.37]),
   'bias': array([0.64])},
  'node_2': {'weights': array([0.37, 0.5 , 0.47]), 'bias': array([0.72])}},
 'layer_3': {'node_1': {'weights': array([0.72, 0.96]), 'bias': array([0.6])},
  'node_2': {'weights': array([0.47, 0.19]), 'bias': array([0.63])},
  'node_3': {'weights': array([0.31, 0.52]), 'bias': array([0.75])}},
 'output': {'node_1': {'weights': array([0.49, 0.18, 0.61]),
   'bias': array([0.02])}}}

The weighted sum at each node is computed as the dot product of the inputs and the weights plus the bias. So let's create a function called *compute_weighted_sum* that does just that.


In [15]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

In [16]:
# Let's generate 5 inputs that we can feed to **small_network**.
from random import seed

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


In [17]:
#Use the compute_weighted_sum function to compute the weighted sum at the first node in the first hidden layer
node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']

weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias)
print('The weighted sum at the first node in the hidden layer is {}'.format(np.around(weighted_sum[0], decimals=4)))


The weighted sum at the first node in the hidden layer is 1.3924


##### Compute Node Activation



Recall that the output of each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.


In [18]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

In [19]:
node_output  = node_activation(compute_weighted_sum(inputs, node_weights, node_bias))
print('The output of the first node in the hidden layer is {}'.format(np.around(node_output[0], decimals=4)))

The output of the first node in the hidden layer is 0.801


##### Forward Propagation

The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the *compute_weighted_sum* and *node_activation* functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.

The way we are going to accomplish this is through the following procedure:

1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
5. Repeat steps 2 - 4 until we compute the output of the output layer.


In [20]:
def forward_propagate(network, inputs):
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer]
        
        layer_outputs = [] 
        for layer_node in layer_data:
        
            node_data = layer_data[layer_node]
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

In [21]:
# Use the forward_propagate function to compute the prediction of our small network
predictions = forward_propagate(small_network, inputs)
print('The predicted value by the network for the given input is {}'.format(np.around(predictions[0], decimals=4)))


The outputs of the nodes in hidden layer number 1 is [0.801, 0.783, 0.7665]
The outputs of the nodes in hidden layer number 2 is [0.8142, 0.8542]
The outputs of the nodes in hidden layer number 3 is [0.8815, 0.764, 0.8095]
The predicted value by the network for the given input is 0.7471


Let's create a new network with;

- 8 inputs,

- 2 Hidden layers with 5 Nodes each, and

- the output layer with a single node

In [23]:
my_network = initialize_network(8, 2, [5, 5], 1) # used the initial_network function to create the network structure

inputs = np.around(np.random.uniform(size=8), decimals=2) # using numpy random function to generate the inputs
print(inputs)

predictions = forward_propagate(my_network, inputs) # use the forward propagate function to print the outputs of the various nodes

print('The predicted values by the network for the given input are {}'.format(predictions))

[0.97 0.74 0.52 0.16 0.95 0.92 0.88 0.05]
The outputs of the nodes in hidden layer number 1 is [0.9461, 0.9321, 0.9528, 0.9646, 0.774]
The outputs of the nodes in hidden layer number 2 is [0.9666, 0.943, 0.9375, 0.9344, 0.9604]
The predicted values by the network for the given input are [0.9391]
